# スコアの妥当性検証

## モデルの検証

### クロンバックのα

:::{card} クロンバックのα

$$
\alpha=\frac{J}{J-1}\left(1-\frac{\sum_{j=1}^J s_j^2}{s_y^2}\right)
$$

:::

- ❌️CTTなので標本依存性がある
- ❌️CTTにおける信頼性係数や、その推定量であるクロンバックのαは項目数が増えるとαも上がってしまい、精度が低いことが知られている


### 適合度

- ❌️IRTの個人適合度（$z^3$統計量） → 正規分布に従い、例えば5%有意水準で適合度が低い受験者を検出できるが、適合していても5%は誤検出する
- ❌️IRTの項目適合度 → 受験者数が増えるとp-valueが5%未満（→適合していない）になりやすく、誤検出が多い



### 情報量関数

:::{card}

- 項目情報関数 $I_j(\theta)=\frac{\left\{P_j^{\prime}(\theta)\right\}^2}{P_j(\theta) Q_j(\theta)}$
- テスト情報関数 $I(\theta) = \sum_{j=1}^J I_j(\theta)$

項目情報関数は フィッシャー情報量（対数尤度関数の2回微分のマイナスの期待値） から導かれ、クラメール・ラオの不等式

$$
I(\theta) \geq \frac{1}{\operatorname{Var}(\hat{\theta})}
$$

より、推定量の分散の下限を示す
:::

情報量関数は1つのテストについて検定のような良し悪しを判断する使われ方はされず、$\theta$の値に対する情報量の多寡をみる

また2つのテスト$A,B$のどちらがより精度が高いかを相対効率$\frac{I_A(\theta)}{I_B(\theta)}$で示すなどといった使われ方をされる。





### $Q_3$統計量

$Q_3$統計量は局所独立性の検証に使われる統計量

:::{admonition}

$Q_3$統計量は任意の項目のペア$j$と$j^\prime$の間の局所独立性の指標として

$$
Q_{3, j j^{\prime}} := r(d_j, d_{j^{\prime}})
$$

と定義される。

- $r(\cdot, \cdot)$：相関係数
- $d_{ij} := u_{ij} - \hat{P}_j(\hat{\theta}_i)$ ：推定値によるICC$\hat{P}_j(\hat{\theta}_i)$と実測値$u_{ij}$の残差得点

:::

$Q_3$統計量は

- $\theta$によって項目間の相関関係が十分に説明されているとすれば、その影響を除去した残差得点同士の相関は0に近くなるはず
- $\theta$以外に項目間に相関をもたらす要因がある場合は$\theta$の影響を除いてもなお相関が残る

という考え方をとっている。

目安としては、「$Q_3 > 0.20$の項目ペアは局所独立性の侵害を疑う必要がある」とされる。

## スコアの検証

### Messickの妥当性概念

| 側面（facet）                      | 内容（簡潔な説明）                                           |
| ---------------------------------- | ------------------------------------------------------------ |
| ① 内容的妥当性（Content）          | 測定内容が理論的構成概念をどれだけ網羅しているか             |
| ② 構成概念妥当性（Substantive）    | 測定が理論に基づき、認知的に妥当か                           |
| ③ 構造的妥当性（Structural）       | 得点構造が想定した構成概念と整合しているか（因子構造など）   |
| ④ 外的妥当性（External）           | 他の指標との関係が理論通りか（収束的・弁別的妥当性）         |
| ⑤ 結果的妥当性（Consequential）    | 測定の利用が社会的に望ましい結果をもたらすか（公正性・偏りなど） |
| ⑥ 実証的妥当性（Generalizability） | 測定結果が一般化可能か（状況・集団を超えて妥当か）           |

### 外的妥当性の検証

スコアが「人間の肌感覚」など何らかの先行的に存在する指標を模倣している場合は、その模倣具合からある程度妥当性を検証できる

例えば数件～数十件の受験者をサンプリングし、人間の肌感覚に従って昇順にソートしてもらい、それと推定された$\hat{\theta}$の順序相関を確認する

## 参考

- [村山航. (2012). 妥当性概念の歴史的変遷と心理測定学的観点からの考察. 教育心理学年報, 51, 118-130.](https://www.jstage.jst.go.jp/article/arepj/51/0/51_118/_pdf)